## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The example uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [1]:
import os

os.environ["AWS_PROFILE"] = "mlflow-profile"
os.environ["AWS_DEFAULT_REGION"] = "eu-north-1"
os.environ["AWS_REGION"] = "eu-north-1"


In [5]:
import mlflow

TRACKING_SERVER_HOST = "ec2-13-60-58-111.eu-north-1.compute.amazonaws.com" # fill in wih the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [3]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-13-60-58-111.eu-north-1.compute.amazonaws.com/:5000'


In [6]:
mlflow.search_experiments() # list_experiments API has been removed, you can use search_experiments instead.()

[<Experiment: artifact_location='s3://priyan-mlflow-artifacts/2', creation_time=1752052909144, experiment_id='2', last_update_time=1752052909144, lifecycle_stage='active', name='priyan', tags={}>,
 <Experiment: artifact_location='s3://priyan-mlflow-artifacts/1', creation_time=1752044883253, experiment_id='1', last_update_time=1752044883253, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://priyan-mlflow-artifacts/0', creation_time=1752040541454, experiment_id='0', last_update_time=1752040541454, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, name="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/07/29 16:15:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/29 16:15:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 's3://priyan-mlflow-artifacts/1/7a3e92af3fe647aa86fa959d16b15d34/artifacts'
🏃 View run youthful-croc-39 at: http://ec2-13-60-58-111.eu-north-1.compute.amazonaws.com:5000/#/experiments/1/runs/7a3e92af3fe647aa86fa959d16b15d34
🧪 View experiment at: http://ec2-13-60-58-111.eu-north-1.compute.amazonaws.com:5000/#/experiments/1


In [ ]:
mlflow.search_experiments()

In [8]:
import joblib
model = joblib.load("liner_reg.bin")

In [9]:
model

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [12]:
mlflow.end_run()

🏃 View run gifted-dove-638 at: http://ec2-13-60-58-111.eu-north-1.compute.amazonaws.com:5000/#/experiments/1/runs/4ffe732c4a044d578edcc4875648d84c
🧪 View experiment at: http://ec2-13-60-58-111.eu-north-1.compute.amazonaws.com:5000/#/experiments/1


In [13]:
with mlflow.start_run():
    mlflow.sklearn.log_model(model, name="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/07/29 16:28:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 's3://priyan-mlflow-artifacts/1/42c7e4e0ca994916a167ab93c7338fab/artifacts'
🏃 View run luxuriant-owl-926 at: http://ec2-13-60-58-111.eu-north-1.compute.amazonaws.com:5000/#/experiments/1/runs/42c7e4e0ca994916a167ab93c7338fab
🧪 View experiment at: http://ec2-13-60-58-111.eu-north-1.compute.amazonaws.com:5000/#/experiments/1


### Interacting with the model registry

In [1]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

ModuleNotFoundError: No module named 'mlflow'

In [15]:
client.search_registered_models()

[]

In [16]:
run_id = client.search_runs(experiment_ids=['1'])[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='ride-predictor'
)

Successfully registered model 'ride-predictor'.
2025/07/29 16:36:07 WARNING mlflow.tracking._model_registry.fluent: Run with id 42c7e4e0ca994916a167ab93c7338fab has no artifacts at artifact path 'models', registering model based on models:/m-366563e04bbf4d24915e553130bda266 instead
2025/07/29 16:36:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ride-predictor, version 1
Created version '1' of model 'ride-predictor'.


<ModelVersion: aliases=[], creation_timestamp=1753806967979, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1753806967979, metrics=None, model_id=None, name='ride-predictor', params=None, run_id='42c7e4e0ca994916a167ab93c7338fab', run_link='', source='models:/m-366563e04bbf4d24915e553130bda266', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [17]:
run_id

'42c7e4e0ca994916a167ab93c7338fab'

In [19]:
logged_model =  f"runs:/{run_id}/models"
model = mlflow.pyfunc.load_model(logged_model)

In [20]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: s3://priyan-mlflow-artifacts/1/models/m-366563e04bbf4d24915e553130bda266/artifacts
  flavor: mlflow.sklearn
  run_id: 42c7e4e0ca994916a167ab93c7338fab